In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import re
from fuzzywuzzy import fuzz



### Discount Rate Estimation

In [6]:
zones = ["", "China","emerg","Europe","Global","India","Japan","Rest"]

def process_df(x):
    columns = x.iloc[0,:]
    columns.name = ""
    x.columns = columns
    x = x.iloc[1:,:]
    x.reset_index(drop=True, names=columns, inplace=True)
    x.rename(index={"6":""}).head(7)
    return x

def get_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_damodaran/totalbeta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_unleveraged_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_leverage/beta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_taxrates(zone=""):
    path = "data_damodaran/discount_rate_estimation/tax/tax_rate_country/taxrate" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry name")
    return x

def get_wacc(zone=""):
    path = "data_damodaran/discount_rate_estimation/wacc_damodaran/wacc" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_10y_tbills():
    path = "data_damodaran/discount_rate_estimation/treasury_bills/histretSP.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["T. Bond yield & return"]
    x = x.iloc[5:,:]
    x = process_df(x)
    return x

def get_10y_tbills():
    path = "data_damodaran/discount_rate_estimation/treasury_bills/histretSP.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["T. Bond yield & return"]
    x = x.iloc[5:,:]
    x = process_df(x)
    return x


def get_risk_premium():
    path = "data_damodaran/discount_rate_estimation/risk_premium/ctrypremJuly23.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["ERPs by country"]
    x = x.iloc[5:,:]
    x = process_df(x)
    return x

#get_risk_premium()

#gics.SubIndustry

In [29]:
#pd.DataFrame(get_wacc())["Tax Rate"].to_csv("/Users/leonplaza/Desktop/tax.csv")

get_betas().sum()


Number of firms                             19979
Average Unlevered Beta                  97.044147
Average Levered Beta                    118.66254
Average correlation with the market     30.956258
Total Unlevered Beta                   325.334125
Total Levered Beta                     396.076429
dtype: object

In [15]:
pd.DataFrame(get_wacc(zone="")["Beta"]).to_csv("/Users/leonplaza/Desktop/beta.csv")

In [339]:
gics = pd.read_csv("Database/gics-map-2018.csv").iloc[:,[1,3,5,7]]
industries = list(get_betas().index)

match = []
for i in industries:
    ls = []
    for k in list(gics.SubIndustry):
        ls.append([fuzz.ratio(i, k),i,k])
    x = pd.DataFrame(ls).sort_values(0, ascending=False)
    match.append(list(x.iloc[0,:]))
        
match = pd.DataFrame(match, columns=["fuzz subind", "damodaran", "subindustry"])
match.set_index("damodaran",inplace=True)
match["tup subind"] = [(match.iloc[i,0],match.iloc[i,1]) for i in range(len(match))]
df_subind = match["tup subind"]

In [340]:
match = []
for i in industries:
    ls = []
    for k in list(gics.Industry):
        ls.append([fuzz.ratio(i, k),i,k])
    x = pd.DataFrame(ls).sort_values(0, ascending=False)
    match.append(list(x.iloc[0,:]))
        
match = pd.DataFrame(match, columns=["fuzz ind", "damodaran", "industry"])
match.set_index("damodaran",inplace=True)
match["tup industry"] = [(match.iloc[i,0],match.iloc[i,1]) for i in range(len(match))]
df_ind = match["tup industry"]

In [341]:
match = []
for i in industries:
    ls = []
    for k in list(gics.IndustryGroup):
        ls.append([fuzz.ratio(i, k),i,k])
    x = pd.DataFrame(ls).sort_values(0, ascending=False)
    match.append(list(x.iloc[0,:]))
        
match = pd.DataFrame(match, columns=["fuzz indgroup", "damodaran", "industry group"])
match.set_index("damodaran", inplace=True)
match["tup indgroup"] = [(match.iloc[i,0],match.iloc[i,1]) for i in range(len(match))]
df_indgroup = match["tup indgroup"]

In [1]:
x = pd.concat([df_subind, df_ind, df_indgroup],axis=1)
ls = []

for i in industries:
    a = [x.loc[i,"tup subind"][0],x.loc[i,"tup industry"][0],x.loc[i,"tup indgroup"][0]]
    ls.append([i,a])


NameError: name 'pd' is not defined

In [360]:
def get_industry(ticket):
    ls = []
    ind = yf.Ticker(ticket).info.get("industry")
    for i in industries:
        ls.append([fuzz.ratio(ind, i),i])
    return pd.DataFrame(ls).sort_values(0, ascending=False).iloc[0,1]

In [361]:
%%time 
get_industry("MSFT")
#yf.Ticker("AAPL").info.get("industry")

CPU times: user 55.5 ms, sys: 5.59 ms, total: 61.1 ms
Wall time: 483 ms


'Software (Internet)'

In [359]:
%%time
ls = []
for i in industries:
    ls.append([fuzz.ratio(yf.Ticker("MSFT").info.get("industry"), i),i])
pd.DataFrame(ls).sort_values(0, ascending=False).iloc[0,1]


CPU times: user 592 ms, sys: 83.5 ms, total: 676 ms
Wall time: 4.37 s


'Software (Internet)'

In [92]:
sp500 = pd.read_excel("data_damodaran/discount_rate_estimation/risk_premium/histimpl.xls")
sp500 = process_df(sp500.iloc[5:,:])
sp500



,Year,Earnings Yield,Dividend Yield,S&P 500,Earnings*,Dividends*,Dividends + Buybacks,Change in Earnings,Change in Dividends,T.Bill Rate,T.Bond Rate,Bond-Bill,Smoothed Growth,Implied Premium (DDM),Analyst Growth Estimate,Implied ERP (FCFE),Implied Premium (FCFE with sustainable Payout),ERP/Riskfree Rate
0,1960,0.0534,0.0341,58.11,3.103074,1.981551,NaN,NaN,NaN,0.0266,0.0276,0.001,0.024484,NaN,NaN,NaN,NaN,NaN
1,1961,0.0471,0.0285,71.55,3.370005,2.039175,NaN,0.086021,0.02908,0.0213,0.0235,0.0022,0.024051,0.0292,NaN,0.0292,NaN,1.242553
2,1962,0.0581,0.034,63.1,3.66611,2.1454,NaN,0.087865,0.052092,0.0273,0.0385,0.0112,0.040496,0.0356,NaN,0.0356,NaN,0.924675
3,1963,0.0551,0.0313,75.02,4.133602,2.348126,NaN,0.127517,0.094493,0.0312,0.0414,0.0102,0.049635,0.0338,NaN,0.0338,NaN,0.816425
4,1964,0.0562,0.0305,84.75,4.76295,2.584875,NaN,0.152252,0.100825,0.0354,0.0421,0.0067,0.051323,0.0331,NaN,0.0331,NaN,0.786223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,NaN,Period,ERP,ERP + Riskfree Rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,1960-2021,0.042131,0.100649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,NaN,2002-2021,0.04979,0.080435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,NaN,2012-2021,0.05353,0.07504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
risk_premium_country = pd.read_excel("data_damodaran/discount_rate_estimation/risk_premium/ctrypremJuly23.xlsx", sheet_name=None)
risk_premium_country = risk_premium_country["ERPs by country"]
loc1 = risk_premium_country[risk_premium_country.iloc[:,0] == "Abu Dhabi"].index.values[0] - 1
loc2 = risk_premium_country[risk_premium_country.iloc[:,0] == "Zambia"].index.values[0] + 1
risk_premium_country = risk_premium_country.iloc[loc1:loc2,:-2]
risk_premium_country = process_df(risk_premium_country)
risk_premium_country

/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Country,Africa,Moody's rating,Rating-based Default Spread,Total Equity Risk Premium,Country Risk Premium,"Sovereign CDS, net of US",Total Equity Risk Premium2,Country Risk Premium3
0,Abu Dhabi,Middle East,Aa2,0.005281,0.05749,0.00749,0.0027,0.053829,0.003829
1,Albania,Eastern Europe & Russia,B1,0.048154,0.118298,0.068298,NaN,NaN,NaN
2,Andorra (Principality of),Western Europe,Baa2,0.020368,0.078888,0.028888,NaN,NaN,NaN
3,Angola,Africa,B3,0.069528,0.148613,0.098613,0.1004,0.192399,0.142399
4,Argentina,Central and South America,Ca,0.128369,0.232068,0.182068,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
152,Uruguay,Central and South America,Baa2,0.020368,0.078888,0.028888,0.0052,0.057375,0.007375
153,Uzbekistan,Eastern Europe & Russia,Ba3,0.038473,0.104567,0.054567,NaN,NaN,NaN
154,Venezuela,Central and South America,C,0.175,0.298205,0.248205,0.1114,0.208,0.158
155,Vietnam,Asia,Ba2,0.032187,0.095651,0.045651,0.013,0.068438,0.018438


In [5]:
pd.read_excel("data_damodaran/discount_rate_estimation/treasury_bills/histretSP.xls", sheet_name=None).keys()

dict_keys(['Explanations and FAQ', 'Returns by year', 'Home Prices', 'S&P 500 & Raw Data', 'T. Bond yield & return', 'T. Bill rates', 'Inflation Rate', 'Summary for ppt', 'Home Prices (Raw Data)', "Moody's Rates", 'Gold Prices', 'Sheet10', 'Sheet11', 'Sheet12', 'Sheet13', 'Sheet14', 'Sheet15', 'Sheet16'])

### Cash Flow Estimation

In [17]:
def get_capex(zone=""):
    path = "data_damodaran/cash_flow_estimation/CAPEX/capex" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_working_capital(zone=""):
    path = "data_damodaran/cash_flow_estimation/working_capital/wcdata" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_margins(zone=""):
    path = "data_damodaran/cash_flow_estimation/Margin/margin" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_financing_flows(zone=""):
    path = "data_damodaran/cash_flow_estimation/Financing_Flows/finflows" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_goodwill(zone=""):
    path = "data_damodaran/cash_flow_estimation/Goodwill/goodwill" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

## Growth Estimations

In [18]:
def get_growth_ebit(zone=""):
    path = "data_damodaran/growth/growth_ebit/fundgrEB" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_growth_eps(zone=""):
    path = "data_damodaran/growth/growth_eps/fundgr" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_growth_roe(zone=""):
    path = "data_damodaran/growth/ROE/roe" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_growth_historical(zone=""):
    path = "data_damodaran/growth/historical_growth/histgr" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x


## Multiples

In [5]:
def get_multiple_ebitda(zone=""):
    path = "data_damodaran/multiples/ebitda/vebitda" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_multiple_pe(zone=""):
    path = "data_damodaran/multiples/peg/pe" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_multiple_roe(zone=""):
    path = "data_damodaran/multiples/roe/pbv" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_multiple_sales(zone=""):
    path = "data_damodaran/multiples/sales/ps" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

### Return Measures

In [6]:
def get_eva(zone=""):
    path = "data_damodaran/return_measures/EVA" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

In [28]:
usa_eva = get_eva(zones[0])
china_eva = get_eva(zones[1])
emerg_eva = get_eva(zones[2])
europe_eva = get_eva(zones[3])
global_eva = get_eva(zones[4])
india_eva = get_eva(zones[5])
rest_eva = get_eva(zones[6])

In [29]:
x = yf.Ticker("MSFT")
x.get_shares_full(start="2020-01-01", end=None)
x.incomestmt


## Capital Structure

In [71]:
x = pd.read_excel("data_damodaran/capital_structure/spread_classifiers/ratings.xls", sheet_name=None)
x = x["Start here Ratings sheet"]
moody = process_df(x.iloc[16:32,:4])
moody_financefirms = process_df(x.iloc[16:,5:9])
moody_smallfirms = process_df(x.iloc[35:,:4])

In [70]:
def get_spread_classifier(size=5e9, financial=False):
    path = "data_damodaran/capital_structure/spread_classifiers/ratings.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Start here Ratings sheet"]
    moody = process_df(x.iloc[16:32,:4])
    moody_financefirms = process_df(x.iloc[16:,5:9])
    moody_smallfirms = process_df(x.iloc[35:,:4])
    
    if financial == True:
        return moody_financefirms
    else:
        if size >= 5e9:
            return moody
        else: 
            return moody_smallfirms

def get_moody_spread(x, size=5e9, financial=False):
    for i in range(len(get_spread_classifier(size,financial=financial))):
        if x <= get_spread_classifier(size,financial=financial).iloc[i,1]:
            return get_spread_classifier(size,financial=financial).iloc[i,-1]
        else:
            continue
    return get_spread_classifier(size,financial=financial).iloc[-1,-1]

,>,≤ to,Rating is,Spread is
0,-100000,0.199999,D2/D,0.16
1,0.2,0.649999,C2/C,0.14
2,0.65,0.799999,Ca2/CC,0.1281
3,0.8,1.249999,Caa/CCC,0.0943
4,1.25,1.499999,B3/B-,0.0604
5,1.5,1.749999,B2/B,0.0435
6,1.75,1.999999,B1/B+,0.0381
7,2,2.25,Ba2/BB,0.0274
8,2.25,2.49999,Ba1/BB+,0.022
9,2.5,2.999999,Baa2/BBB,0.0189


In [13]:
def drop_identical_columns(df):
    columns_to_drop = []
    for i in range(len(df.columns)):
        for j in range(i+1, len(df.columns)):
            if df.iloc[:, i].equals(df.iloc[:, j]):
                columns_to_drop.append(df.columns[j])
    df.drop(columns=columns_to_drop, inplace=True)
    return df



def columns_for_sql(df):
    column = pd.Series(df.columns).apply(lambda x: str(x).lower().replace(" ","_"))
    column = column.apply(lambda x: re.sub("[^A-Za-z0-9|_]","",x))
    column = column.apply(lambda x: x.replace("__","_"))
    column = list(column)
    return column


                
def table_by_zone(x):
    x = pd.concat([get_betas(x), get_capex(x), get_eva(x), get_financing_flows(x), get_goodwill(x),
               get_growth_ebit(x), get_growth_eps(x), get_growth_historical(x), get_growth_roe(x),
               get_margins(x), get_multiple_ebitda(x), get_multiple_pe(x), get_multiple_roe(x), 
               get_multiple_sales(x), get_taxrates(x), get_unleveraged_betas(x), get_wacc(x), 
               get_working_capital(x)], axis=1)
    x = x.dropna(how="all", axis=1)
    x = x.T.drop_duplicates().T
    x = drop_identical_columns(x)
    #x.columns = columns_for_sql(x)
    #x.reset_index()
    return x


def discount_rate_by_zone(x):
    x = pd.concat([get_wacc(x), get_betas(x), get_unleveraged_betas(x), get_taxrates(x)], axis=1)
    x = x.T.drop_duplicates().T
    x = drop_identical_columns(x)
    x = x.dropna(how="all", axis=1)
    x.columns = columns_for_sql(x)
    x.reset_index()
    return x


def growth_by_zone(x):
    x = pd.concat([get_growth_ebit(x), get_growth_eps(x), get_growth_historical(x), get_growth_roe(x)], axis=1)
    x = x.T.drop_duplicates().T
    x = drop_identical_columns(x)
    x = x.dropna(how="all", axis=1)
    x.columns = columns_for_sql(x)
    x.reset_index()
    return x


def cash_flow_by_zone():
    x = pd.concat([get_capex(x), get_financing_flows(x), get_goodwill(x), 
                   get_working_capital(x), get_margins], axis=1)
    x = x.T.drop_duplicates().T
    x = drop_identical_columns(x)
    x = x.dropna(how="all", axis=1)
    x.columns = columns_for_sql(x)
    x.reset_index()
    return x

In [34]:
table_by_zone("").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/usa.csv")
table_by_zone("China").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/china.csv")
table_by_zone("Europe").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/europe.csv")
table_by_zone("India").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/india.csv")
table_by_zone("Japan").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/japan.csv")
table_by_zone("Rest").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/CAN-AUS-NZ.csv")
table_by_zone("Global").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/global.csv")

In [35]:
discount_rate_by_zone("").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/usa_discount.csv")
discount_rate_by_zone("China").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/china_discount.csv")
discount_rate_by_zone("Europe").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/europe_discount.csv")
discount_rate_by_zone("India").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/india_discount.csv")
discount_rate_by_zone("Japan").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/japan_discount.csv")
discount_rate_by_zone("Rest").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/CANAUSNZ_discount.csv")
discount_rate_by_zone("Global").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/global_discount.csv")
    

In [16]:
usa = table_by_zone("")
"free" in usa.columns[0]

False

In [23]:
ls = []
x = usa.columns
for i in x:
    ls.append("risk" in str(i).lower())
    
x[ls]

Index(['HiLo Risk'], dtype='object', name='')

In [39]:
def pv_fcff(t):
    ebit*(1-tax) + depr[t] - capex[t] + ktn_change[t]

In [94]:
#msft.incomestmt#.loc["Reconciled Depreciation",:].sort_index(ascending=True)


In [53]:
msft.cashflow

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Free Cash Flow,99584000000.0,111443000000.0,92953000000.0,73365000000.0
Repurchase Of Capital Stock,-77550000000.0,-89402000000.0,-85971000000.0,-72358000000.0
Repayment Of Debt,-11151000000.0,-9543000000.0,-8750000000.0,-13592000000.0
Issuance Of Debt,5228000000.0,9420000000.0,20393000000.0,16091000000.0
Issuance Of Capital Stock,NaN,NaN,1105000000.0,880000000.0
Capital Expenditure,-10959000000.0,-10708000000.0,-11085000000.0,-7309000000.0
Interest Paid Supplemental Data,3803000000.0,2865000000.0,2687000000.0,3002000000.0
Income Tax Paid Supplemental Data,18679000000.0,19573000000.0,25385000000.0,9501000000.0
End Cash Position,30737000000.0,24977000000.0,35929000000.0,39789000000.0
Beginning Cash Position,24977000000.0,35929000000.0,39789000000.0,50224000000.0


In [55]:
msft.balancesheet

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Treasury Shares Number,0.0,NaN,NaN,NaN
Ordinary Shares Number,15550061000.0,15943425000.0,16426786000.0,16976763000.0
Share Issued,15550061000.0,15943425000.0,16426786000.0,16976763000.0
Net Debt,81123000000.0,96423000000.0,89779000000.0,74420000000.0
Total Debt,111088000000.0,120069000000.0,124719000000.0,112436000000.0
Tangible Book Value,62146000000.0,50672000000.0,63090000000.0,65339000000.0
Invested Capital,173234000000.0,170741000000.0,187809000000.0,177775000000.0
Working Capital,-1742000000.0,-18577000000.0,9355000000.0,38321000000.0
Net Tangible Assets,62146000000.0,50672000000.0,63090000000.0,65339000000.0
Common Stock Equity,62146000000.0,50672000000.0,63090000000.0,65339000000.0
